# Multi-label recognition


### Sources:
- Humans
- Google
- Microsoft Azure
- Amazon
- Detectron2

In [1]:
import pickle
import os
import json

In [2]:
multi_label_compiled = {}

##### Microsoft Azure Responses

In [3]:
with open('azure_detected_tags_full.p','rb') as F:
    azure_detected_tags = pickle.load(F)

In [4]:
for im_path, tag_info in azure_detected_tags.items():
    im_name = im_path.split('/')[-1]
    tags = [i[0] for i in tag_info[:3]]
    
    if not im_name in multi_label_compiled.keys():
        multi_label_compiled[im_name] = {}
    
    multi_label_compiled[im_name]['azure'] = tags

##### Amazon Rekognition Responses

In [5]:
with open('rekognition_detected_tags.p','rb') as F:
    rekognition_detected_tags = pickle.load(F)

In [6]:
# len(rekognition_detected_tags.keys())

In [7]:
for im_path, tag_info in rekognition_detected_tags.items():
    im_name = im_path.split('/')[-1]
    tags = [i[0] for i in tag_info[:3]]
    if not im_name in multi_label_compiled.keys():
        multi_label_compiled[im_name] = {}
    multi_label_compiled[im_name]['rekognition'] = tags

##### Google Cloud Vision API Responses

In [8]:
for im in os.listdir('static/dataset/api_outputs/'):
    im_info_path = os.path.join('static/dataset/api_outputs/', im)
    with open(im_info_path,'rb') as F:
        im_info = pickle.load(F)
    tags = im_info[0][:3]
    im_name = im.replace('p','jpg')
    
    if not im_name in multi_label_compiled.keys():
        multi_label_compiled[im_name] = {}
    
    multi_label_compiled[im_name]['google'] = tags

##### Detectron2

In [9]:
with open('detectron_predictions.p','rb') as F:
    detectron_predictions = pickle.load(F)

In [10]:
for im_path in detectron_predictions.keys():
    im_name = im_path.split('/')[-1]
    multi_label_compiled[im_name]['detectron'] = detectron_predictions[im_path]

In [11]:
# multi_label_compiled[im_name]['detectron']

##### Human Response

In [12]:
with open('human_or_ai.p','rb') as F:
    human_or_ai = pickle.load(F)

In [13]:
# human_or_ai

In [14]:
with open('response_structure_labels.json','r') as F:
    response_structure_labels = json.load(F)

human_resps = []
for key, values in response_structure_labels.items():
    im_name = key.split('/')[-1]
    human_resps.append(im_name)
    multi_label_compiled[im_name]['humans'] = []
    for valkey in values.keys():
        if human_or_ai[valkey] == 'H':
            valval = values[valkey]
            multi_label_compiled[im_name]['humans'].append(valval)
        else:
            pass

In [15]:
multi_label_compiled_with_humans = {}

In [16]:
ct = 0
for im_name in multi_label_compiled.keys():
    if 'humans' in multi_label_compiled[im_name].keys():
        if len(multi_label_compiled[im_name]['detectron']) == 0:
            print(im_name)
        else:
            multi_label_compiled_with_humans[im_name] = multi_label_compiled[im_name]

2337.jpg
264.jpg
379.jpg
385.jpg
901.jpg


In [17]:
for im_name in multi_label_compiled_with_humans.keys():
    if len(multi_label_compiled_with_humans[im_name]['detectron']) == 0:
        print(im_name)

In [18]:
len(multi_label_compiled_with_humans.keys())

808

In [19]:
multi_label_compiled_with_humans['10.jpg']

{'azure': ['clothing', 'person', 'outdoor'],
 'rekognition': ['Shorts', 'Clothing', 'Apparel'],
 'google': ['Shorts', 'Active shorts', 'Outdoor recreation'],
 'detectron': ['person', 'tennis racket', 'handbag'],
 'humans': [['children', 'bat', 'ground']]}

In [20]:
image_nums = [i.split('.')[0] for i in list(multi_label_compiled_with_humans.keys())]

In [21]:
len(image_nums)

808

In [22]:
len(multi_label_compiled_with_humans.keys())

808

In [27]:
# with open('multi_label_compiled.p','wb') as F:
#     pickle.dump(multi_label_compiled,F)

In [28]:
# with open('multi_label_compiled_with_humans.p','wb') as F:
#     pickle.dump(multi_label_compiled_with_humans,F)

# Create Response HTMLs

In [2]:
import pickle

In [3]:
string_1 = "<center> <h4>Question: What do you see in this image? <br> Speaker Response: Bello Bello Bello</h4> </center>"

In [4]:
# with open('multi_label_compiled_with_humans.p','rb') as F:
#     multi_label_compiled_with_humans = pickle.load(F)

In [5]:
from tqdm import tqdm

In [6]:
agent_to_code = {}
agent_to_code['humans'] = 'a'
agent_to_code['azure'] = 'b'
agent_to_code['detectron'] = 'c'
agent_to_code['google'] = 'd'
agent_to_code['rekognition'] = 'e'

In [49]:
for im_name in tqdm(multi_label_compiled_with_humans.keys()):
    for agent in multi_label_compiled_with_humans[im_name].keys():
        agent_code = agent_to_code[agent]
        responses = multi_label_compiled_with_humans[im_name][agent]
        if type(responses[0]) is list:
            human_ct = 0
            for response in responses:
                file_name = "static/responses/%s_%s_%s.html"%(im_name.split('.jpg')[0],agent_code,human_ct)
                string_new = string_1.replace('Bello Bello Bello',', '.join(response).lower())
                with open(file_name,'w') as F:
                    print(string_new, file = F)
                human_ct += 1
                break
        else:
            human_ct = 0
            file_name = "static/responses/%s_%s_%s.html"%(im_name.split('.jpg')[0],agent_code,human_ct)
            string_new = string_1.replace('Bello Bello Bello',', '.join(responses).lower())
            with open(file_name,'w') as F:
                print(string_new, file = F)

100%|██████████| 808/808 [00:07<00:00, 113.09it/s]


In [50]:
# rl = ['341', '191', '320', '693', '216', '944', '2278', '939', '297', '418', '419', '218', '3003', '957', '392', '359', '207', '550', '23', '275', '506', '45', '409', '493', '3402', '511', '703', '683', '289', '101', '595', '518', '87', '954', '926', '112', '677', '916', '322', '4374', '336', '242', '3770', '126', '374', '79', '722', '805', '283', '772', '179', '133', '697', '3374', '737', '754', '2818', '923', '783', '349', '1136', '280', '320', '538', '407', '701', '762', '3855', '285', '186', '3571', '5', '508', '884', '46', '367', '616', '437', '817', '638', '860', '452', '91', '383', '998', '324', '632', '243', '414', '890', '137', '878', '116', '1080', '583', '865', '159', '870', '702', '2', '3828', '359', '732', '539', '496', '60', '737', '637', '3999', '305', '4250', '278', '32', '945', '936', '727', '961', '2022', '365', '635', '121', '783', '85', '99', '672', '942', '460', '794', '906', '730', '714', '309', '3118', '90', '1931', '48', '529', '441', '310', '4916', '694', '957', '386', '721', '2465', '947', '44', '1200', '383', '584', '381', '410', '2674', '165', '950', '276', '641', '706', '925', '95', '944', '950', '1968', '687', '442', '704', '723', '324', '379', '728', '3958', '4653', '2749', '394', '217', '398', '838', '994', '388', '648', '670', '259', '645', '719', '158', '210', '3703', '606', '172', '253', '58', '928', '939', '796', '311', '994', '2614', '629', '968', '1868', '792', '2982', '497', '128', '4775', '304', '899', '739', '963', '875', '386', '205', '944', '801', '63', '498', '4581', '801', '2155', '3051', '579', '51', '2173', '991', '857', '534', '962', '345', '581', '376', '36', '543', '601', '822', '927', '387', '302', '217', '321', '738', '2959', '1267', '824', '573', '1140', '19', '218', '518', '88', '700', '282', '858', '94', '5', '952', '456', '390', '921', '882', '457', '144', '570', '277', '1621', '1395', '164', '131', '622', '239', '17', '650', '93', '179', '601', '793', '870', '3366', '930', '77', '54', '862', '328', '460', '868', '860', '3917', '3976', '818', '387', '995', '909', '164', '572', '1334', '2428', '818', '1209', '554', '649', '3808', '114', '52', '550', '304', '901', '655', '373', '327', '935', '783', '326', '212', '476', '7', '4123', '18', '82', '426', '4896', '475', '552', '2532', '4158', '331', '558', '268', '104', '30', '152', '942', '694', '442', '548', '78', '168', '609', '1712', '541', '549', '284', '811', '325', '842', '456', '1425', '757', '1701', '224', '580', '273', '4127', '541', '597', '597', '919', '723', '84', '4358', '2387', '775', '354', '1159', '190', '922', '259', '97', '637', '145', '668', '1960', '654', '338', '411', '461', '444', '2642', '143', '3202', '975', '807', '430', '938', '174', '457', '191', '575', '3364', '958', '11', '130', '439', '474', '581', '452', '931', '743', '606', '1936', '256', '459', '230', '813', '843', '3087', '416', '1766', '945', '386', '479', '564', '55', '561', '726', '677', '302', '749', '818', '536', '130', '91', '62', '243', '4319', '537', '532', '1284', '211', '602', '604', '127', '763', '752', '510', '561', '10', '1391', '907', '3182', '909', '728', '494', '951', '386', '669', '503', '440', '513', '88', '396', '734', '100', '1727', '864', '29', '70', '335', '3319', '597', '392', '387', '162', '105', '744', '4687', '766', '471', '612', '588', '460', '691', '810', '16', '894', '3542', '668', '235', '515', '458', '119', '225', '631', '222', '183', '585', '508', '588', '742', '822', '2411', '280', '38', '462', '1139', '446', '865', '564', '574', '52', '945', '42', '242', '560', '995', '386', '570', '930', '360', '670', '4621', '392', '246', '570', '420', '534', '2872', '138', '756', '347', '910', '492', '924', '898', '3562', '3258', '676', '167', '849', '600', '3020', '380', '774', '678', '520', '368', '993', '480', '995', '651', '815', '64', '624', '877', '89', '628', '3541', '637', '599', '225', '27', '777', '587', '915', '2870', '99', '267', '681', '34', '996', '369', '3272', '503', '173', '2638', '3716', '2970', '830', '3319', '385', '311', '328', '661', '952', '664', '599', '281', '236', '386', '451', '229', '657', '889', '331', '648', '4593', '87', '467', '2187', '982', '514', '314', '873', '124', '289', '887', '242', '369', '203', '996', '4418', '120', '368', '1333', '87', '4056', '925', '578', '33', '2276', '71', '616', '370', '252', '573', '457', '2659', '946', '340', '4089', '3204', '258', '454', '580', '1691', '497', '835', '870', '808', '76', '573', '88', '263', '3323', '689', '461', '776', '594', '387', '843', '2227', '1564', '491', '117', '510', '219', '298', '993', '3393', '374', '794', '331', '774', '393', '2353', '241', '227', '664', '839', '2138', '113', '622', '587', '230', '208', '637', '613', '39', '237', '352', '917', '863', '194', '346', '898', '210', '92', '704', '901', '954', '4415', '3248', '802', '173', '612', '4316', '455', '856', '65', '347', '734', '371', '1736', '34', '4787', '3555', '494', '761', '306', '949', '828', '434', '639', '647', '74', '760', '785', '720', '70', '1725', '114', '394', '462', '35', '649', '852', '516', '910', '380', '889', '823', '77', '499', '769', '7', '234', '915', '130', '315', '3878', '966', '65', '849', '728', '101', '196', '3266', '154', '1234', '2169', '127', '115', '81', '286', '885', '828', '233', '553', '636', '459', '632', '202', '2312', '612', '855', '708', '405', '989', '369', '325', '507', '240', '448', '528', '1797', '758', '28', '980', '1650', '531', '322', '4704', '908', '826', '266', '374', '45', '791', '507', '990', '905', '133', '974', '2915', '304', '2127', '882', '376', '16', '911', '655', '657', '444', '498', '126', '715', '63', '450', '296', '458', '624', '749', '987', '561', '358', '89', '568', '684', '27', '464', '584', '4707', '863', '351', '981', '908', '993', '819', '757', '792', '3932', '176', '1815', '260', '742', '260', '75', '73', '796', '158', '503', '846', '9', '648', '15', '249', '993', '381', '3805', '980', '4053', '978', '566', '635', '142', '209', '913', '235', '2672', '1443', '320', '692', '3751', '724', '54', '3589', '867', '735', '267', '656', '4871', '176', '259', '184', '1000', '797', '183', '431', '857', '225', '179', '2337', '112', '570', '739', '755', '2557', '1887', '2595', '417', '587', '525', '195', '1249', '236', '12', '608', '511', '803', '571', '204', '2757', '148', '86', '562', '78', '692', '258', '4487', '856', '28', '265', '4212', '731', '807', '700', '123', '240', '483', '294', '461', '723', '74', '471', '580', '787', '516', '644', '748', '103', '4355', '294', '150', '4395', '907', '854', '139', '1652', '55', '4334', '4753', '53', '72', '255', '54', '137', '925', '36', '408', '368', '854', '2753', '786', '509', '932', '96', '35', '160', '884', '100', '636', '312', '494', '123', '919', '297', '2810', '57', '350', '834', '590', '646', '687', '97', '328', '340', '534', '784', '180', '674', '935', '937', '3269', '320', '608', '3898', '488', '769', '134', '29', '866', '997', '281', '1883', '805', '456', '957', '686', '438', '735', '873', '355', '2860', '415', '2546', '764', '676', '152', '996', '19', '224', '1824', '362', '208', '895', '169', '875', '1634', '265', '602', '2526', '35', '150', '565', '745', '91', '874', '1094', '207', '2382', '1065', '322', '92', '742', '874', '420', '801', '717', '924', '946', '4838', '296', '609', '176', '796', '668', '4743', '839', '989', '923', '511', '66', '684', '736', '467', '1282', '352', '534', '331', '126', '501', '970', '828', '868', '1008', '545', '164', '968', '2164', '248', '2744', '494', '324', '947', '180', '375', '4640', '293', '995', '1856', '423', '812', '48', '464', '968', '553', '223', '248', '545', '80', '682', '295', '4478', '294', '2694', '292', '361', '545', '124', '646', '431', '2562', '489', '4798', '28', '943', '231', '725', '1475', '992', '472', '523', '731', '111', '4491', '470', '264', '2131', '670', '126', '2015', '4441', '649', '584', '893', '1669', '872', '348', '247', '696', '136', '648', '910', '488', '841', '744', '3172', '554', '738', '97', '513', '342', '1276', '297', '833', '962', '188', '515', '453', '669', '584', '427', '77', '846', '132', '418', '661', '571', '780', '94', '505']

In [51]:
all_htmls = os.listdir('static/responses/')

In [52]:
human_htmls = []
machine_htmls = []

for html in all_htmls:
    if 'humans' in html:
        human_htmls.append(html)
    else:
        if 'trial_img' not in html:
            machine_htmls.append(html)

In [53]:
len(human_htmls)

0

In [54]:
len(machine_htmls)

4040

In [55]:
big_str = "','".join(image_nums)
print("var = ['%s']"%big_str)

var = ['10','100','1000','1008','101','103','104','105','1065','1080','1094','11','111','112','113','1136','1139','114','1140','115','1159','116','117','119','12','120','1200','1209','121','123','1234','124','1249','126','1267','127','1276','128','1282','1284','130','131','132','133','1333','1334','134','136','137','138','139','1391','1395','142','1425','143','144','1443','145','1475','148','15','150','152','154','1564','158','159','16','160','162','1621','1634','164','165','1650','1652','1669','167','168','169','1691','17','1701','1712','172','1725','1727','173','1736','174','176','1766','179','1797','18','180','1815','1824','183','184','1856','186','1868','188','1883','1887','19','190','191','1931','1936','194','195','196','1960','1968','2','2015','202','2022','203','204','205','207','208','209','210','211','212','2127','2131','2138','2155','216','2164','2169','217','2173','218','2187','219','222','2227','223','224','225','227','2276','2278','229','23','230','231','2312','233','234',